# Part 1 - Identification of Squad Deficiencies

To determine what signings would best suit a club, we need to know where in the current squad the club is lacking in performance. Part One of this project involves identifying these deficiencies, based on how squad players compare with players of other clubs. 

In [1]:
import pandas as pd
import soccerdata as sd

[06/05/25 12:03:51] INFO     No custom team name replacements found. You can configure these in       ]8;id=561885;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=386634;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py#90\90]8;;\
                             /home/jasmine/soccerdata/config/teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=115403;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=54666;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py#196\196]8;;\
                             /home/jasmine/soccerdata/config/league_dict.json.                                     

In [2]:
# 24/25 data
FBREF = sd.FBref(leagues="ENG-Premier League", seasons=2024)
FBREF

[06/05/25 12:04:05] INFO     Saving cached data to /home/jasmine/soccerdata/data/FBref               ]8;id=67592;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=76365;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

The `soccerdata` package allows me to obtain season statistics for each player in the Premier League. I would like to group these players by their position, and select statistics (features) correspondingly for each position.

In [94]:
# grab all relevant statistics
player_stats = FBREF.read_player_season_stats(stat_type="standard")
player_defense = FBREF.read_player_season_stats(stat_type="defense")
player_passing = FBREF.read_player_season_stats(stat_type="passing")
player_shooting = FBREF.read_player_season_stats(stat_type="shooting")
player_keeper = FBREF.read_player_season_stats(stat_type="keeper")

In [5]:
player_stats.head()

nation    pos     age  born  \
                                                                              
league             season team    player                                      
ENG-Premier League 2425   Arsenal Ben White        ENG     DF  27-025  1997   
                                  Bukayo Saka      ENG  MF,FW  23-058  2001   
                                  David Raya       ESP     GK  29-048  1995   
                                  Declan Rice      ENG     MF  25-293  1999   
                                  Ethan Nwaneri    ENG  MF,FW  17-226  2007   

                                                Playing Time                   \
                                                          MP Starts  Min  90s   
league             season team    player                                        
ENG-Premier League 2425   Arsenal Ben White                7      6  585  6.5   
                                  Bukayo Saka              8      8  650  7.2   
                                  David Raya               9      9  810  9.0   
                                  Declan Rice              8      8  672  7.5   
                                  Ethan Nwaneri            4      0   27  0.3   

                                                Performance      ...  \
                                                        Gls Ast  ...   
league             season team    player                         ...   
ENG-Premier League 2425   Arsenal Ben White               0   1  ...   
                                  Bukayo Saka             3   7  ...   
                                  David Raya              0   0  ...   
                                  Declan Rice             0   1  ...   
                                  Ethan Nwaneri           0   0  ...   

                                                Per 90 Minutes              \
                                                           Gls   Ast   G+A   
league             season team    player                                     
ENG-Premier League 2425   Arsenal Ben White                0.0  0.15  0.15   
                                  Bukayo Saka             0.42  0.97  1.38   
                                  David Raya               0.0   0.0   0.0   
                                  Declan Rice              0.0  0.13  0.13   
                                  Ethan Nwaneri            0.0   0.0   0.0   

                                                                          \
                                                 G-PK G+A-PK    xG   xAG   
league             season team    player                                   
ENG-Premier League 2425   Arsenal Ben White       0.0   0.15  0.02  0.03   
                                  Bukayo Saka    0.42   1.38  0.31  0.49   
                                  David Raya      0.0    0.0   0.0   0.0   
                                  Declan Rice     0.0   0.13  0.05  0.17   
                                  Ethan Nwaneri   0.0    0.0  0.07  0.35   

                                                                       
                                                xG+xAG  npxG npxG+xAG  
league             season team    player                               
ENG-Premier League 2425   Arsenal Ben White       0.04  0.02     0.04  
                                  Bukayo Saka      0.8  0.31      0.8  
                                  David Raya       0.0   0.0      0.0  
                                  Declan Rice     0.22  0.05     0.22  
                                  Ethan Nwaneri   0.42  0.07     0.42  

[5 rows x 33 columns]

In [6]:
# unique positions
player_stats["pos"].unique()

<StringArray>
['DF', 'MF,FW', 'GK', 'MF', 'FW,MF', 'FW', 'MF,DF', 'FW,DF', 'DF,MF', 'DF,FW']
Length: 10, dtype: string

Note that some players play in more than one position. There is no way to partition the statistics based on which of the two positions they played, and so there are various approaches I can take here. 

1) Only take the first (primary) position
2) Take both positions (i.e., player will be part of two datasets), but under the pretense that it may skew the data
3) Instead of using simple Forward, Midfield, Defense positions, further split the positions into Attacking Midfielder and Defensive Midfielder

Option 3 seems the most reasonable. The order of the positions does matter though - e.g., "MF,DF" is not the same as "DF,MF." Additionally, some players are specified solely as "MF" even if they are Attacking or Defensive midfielders. **Transfermarkt** seems to have more specific player positions (as well as transfer-specific data, which I'll need later on), but this is only accessible via the R package `worldfootballR`.

In [12]:
player_stats[player_stats["pos"] == "MF"]

nation pos     age  \
                                                                           
league             season team        player                               
ENG-Premier League 2425   Arsenal     Declan Rice        ENG  MF  25-293   
                                      Jorginho           ITA  MF  32-318   
                                      Martin Ødegaard    NOR  MF  25-321   
                          Aston Villa Amadou Onana       BEL  MF  23-078   
                                      Boubacar Kamara    FRA  MF  24-345   
...                                                      ...  ..     ...   
                          Wolves      André              BRA  MF  23-109   
                                      Chiquinho          POR  MF  24-271   
                                      João Gomes         BRA  MF  23-264   
                                      Mario Lemina       GAB  MF  31-062   
                                      Tommy Doyle        ENG  MF  23-016   

                                                       born Playing Time  \
                                                                      MP   
league             season team        player                               
ENG-Premier League 2425   Arsenal     Declan Rice      1999            8   
                                      Jorginho         1991            2   
                                      Martin Ødegaard  1998            3   
                          Aston Villa Amadou Onana     2001            8   
                                      Boubacar Kamara  1999            1   
...                                                     ...          ...   
                          Wolves      André            2001            7   
                                      Chiquinho        2000            1   
                                      João Gomes       2001            9   
                                      Mario Lemina     1993            9   
                                      Tommy Doyle      2001            7   

                                                                        \
                                                      Starts  Min  90s   
league             season team        player                             
ENG-Premier League 2425   Arsenal     Declan Rice          8  672  7.5   
                                      Jorginho             2  150  1.7   
                                      Martin Ødegaard      3  253  2.8   
                          Aston Villa Amadou Onana         8  540  6.0   
                                      Boubacar Kamara      0    9  0.1   
...                                                      ...  ...  ...   
                          Wolves      André                5  363  4.0   
                                      Chiquinho            0    7  0.1   
                                      João Gomes           9  747  8.3   
                                      Mario Lemina         9  764  8.5   
                                      Tommy Doyle          1  204  2.3   

                                                      Performance      ...  \
                                                              Gls Ast  ...   
league             season team        player                           ...   
ENG-Premier League 2425   Arsenal     Declan Rice               0   1  ...   
                                      Jorginho                  0   0  ...   
                                      Martin Ødegaard           0   0  ...   
                          Aston Villa Amadou Onana              2   0  ...   
                                      Boubacar Kamara           0   0  ...   
...                                                           ...  ..  ...   
                          Wolves      André                     0   0  ...   
                                      Chiquinho                 0   0  ...   
                             

In [73]:
tm_player_stats = pd.read_csv("PL2425_player_stats.csv")
tm_player_stats

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
0,1,Manchester City,Premier League,England,Ederson,https://www.transfermarkt.com/ederson/profil/s...,Goalkeeper,31,Brazil,48,37,0,3311
1,2,Manchester City,Premier League,England,Stefan Ortega,https://www.transfermarkt.com/stefan-ortega/pr...,Goalkeeper,32,Germany,57,21,0,1819
2,3,Manchester City,Premier League,England,Scott Carson,https://www.transfermarkt.com/scott-carson/pro...,Goalkeeper,39,England,26,0,0,0
3,4,Manchester City,Premier League,England,Spike Brits,https://www.transfermarkt.com/spike-brits/prof...,Goalkeeper,17,England,1,0,0,0
4,5,Manchester City,Premier League,England,Max Hudson,https://www.transfermarkt.com/max-hudson/profi...,Goalkeeper,17,Wales,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,788,Ipswich Town,Premier League,England,Nathan Broadhead,https://www.transfermarkt.com/nathan-broadhead...,Left Winger,27,Wales,28,21,2,836
788,789,Ipswich Town,Premier League,England,George Hirst,https://www.transfermarkt.com/george-hirst/pro...,Centre-Forward,26,Scotland,29,29,5,833
789,790,Ipswich Town,Premier League,England,Ali Al-Hamadi,https://www.transfermarkt.com/ali-al-hamadi/pr...,Centre-Forward,23,Iraq,15,13,1,276
790,791,Ipswich Town,Premier League,England,Wes Burns,https://www.transfermarkt.com/wes-burns/profil...,Right Winger,30,Wales,22,19,0,979


In [74]:
tm_player_stats['minutes_played'].describe()

count     792.000000
mean     1223.679293
std      1301.703849
min         0.000000
25%        17.750000
50%       775.500000
75%      2277.000000
max      4980.000000
Name: minutes_played, dtype: float64

In [75]:
tm_player_stats500 = tm_player_stats[tm_player_stats["minutes_played"] >= 500]
tm_player_stats500

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
0,1,Manchester City,Premier League,England,Ederson,https://www.transfermarkt.com/ederson/profil/s...,Goalkeeper,31,Brazil,48,37,0,3311
1,2,Manchester City,Premier League,England,Stefan Ortega,https://www.transfermarkt.com/stefan-ortega/pr...,Goalkeeper,32,Germany,57,21,0,1819
5,6,Manchester City,Premier League,England,Josko Gvardiol,https://www.transfermarkt.com/josko-gvardiol/p...,Left-Back,23,Croatia,56,53,6,4513
6,7,Manchester City,Premier League,England,Rúben Dias,https://www.transfermarkt.com/ruben-dias/profi...,Centre-Back,28,Portugal,43,41,0,3215
7,8,Manchester City,Premier League,England,Rico Lewis,https://www.transfermarkt.com/rico-lewis/profi...,Right-Back,20,England,54,43,2,3072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,785,Ipswich Town,Premier League,England,Jack Clarke,https://www.transfermarkt.com/jack-clarke/prof...,Left Winger,24,England,39,36,3,1481
786,787,Ipswich Town,Premier League,England,Conor Chaplin,https://www.transfermarkt.com/conor-chaplin/pr...,Second Striker,28,England,29,23,2,944
787,788,Ipswich Town,Premier League,England,Nathan Broadhead,https://www.transfermarkt.com/nathan-broadhead...,Left Winger,27,Wales,28,21,2,836
788,789,Ipswich Town,Premier League,England,George Hirst,https://www.transfermarkt.com/george-hirst/pro...,Centre-Forward,26,Scotland,29,29,5,833


In [76]:
# the last 3 positions are outlying - see Rmd file
tm_player_stats500["player_pos"].unique()

array(['Goalkeeper', 'Left-Back', 'Centre-Back', 'Right-Back',
       'Central Midfield', 'Attacking Midfield', 'Centre-Forward',
       'Right Winger', 'Left Winger', 'Defensive Midfield',
       'Second Striker', 'Left Midfield', 'Right Midfield'], dtype=object)

In [77]:
# dorgu and j. ramsey - both left backs
tm_player_stats500[tm_player_stats500["player_pos"] == "Left Midfield"]

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
275,276,Manchester United,Premier League,England,Patrick Dorgu,https://www.transfermarkt.com/patrick-dorgu/pr...,Left Midfield,20,Denmark,20,20,0,1522
318,319,Aston Villa,Premier League,England,Jacob Ramsey,https://www.transfermarkt.com/jacob-ramsey/pro...,Left Midfield,24,England,46,45,4,2274


In [78]:
# rodrigo gomes is a right winger
tm_player_stats500[tm_player_stats500["player_pos"] == "Right Midfield"]

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
517,518,Wolverhampton Wanderers,Premier League,England,Rodrigo Gomes,https://www.transfermarkt.com/rodrigo-gomes/pr...,Right Midfield,21,Portugal,38,28,3,1068


In [79]:
tm_player_stats500[tm_player_stats500["player_pos"] == "Second Striker"]

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
126,127,Chelsea FC,Premier League,England,João Félix,https://www.transfermarkt.com/joao-felix/profi...,Second Striker,25,Portugal,28,20,7,948
527,528,Wolverhampton Wanderers,Premier League,England,Matheus Cunha,https://www.transfermarkt.com/matheus-cunha/pr...,Second Striker,26,Brazil,36,36,17,2832
786,787,Ipswich Town,Premier League,England,Conor Chaplin,https://www.transfermarkt.com/conor-chaplin/pr...,Second Striker,28,England,29,23,2,944


In [80]:
tm_player_stats500.loc[tm_player_stats500["player_pos"] == "Second Striker", "player_pos"] = "Centre-Forward"
tm_player_stats500.loc[tm_player_stats500["player_pos"] == "Left Midfield", "player_pos"] = "Left-Back"
tm_player_stats500.loc[tm_player_stats500["player_pos"] == "Right Midfield", "player_pos"] = "Right Winger"

In [81]:
tm_player_stats500["player_pos"].unique()

array(['Goalkeeper', 'Left-Back', 'Centre-Back', 'Right-Back',
       'Central Midfield', 'Attacking Midfield', 'Centre-Forward',
       'Right Winger', 'Left Winger', 'Defensive Midfield'], dtype=object)

I now need to merge the two player stats dataframes that I have. 

In [98]:
# drop the multilevel index
player_stats = player_stats.reset_index()
player_defense = player_defense.reset_index()
player_passing = player_passing.reset_index()
player_shooting = player_shooting.reset_index()
player_keeper = player_keeper.reset_index()
player_stats

league season     team           player nation    pos  \
                                                                         
0    ENG-Premier League   2425  Arsenal        Ben White    ENG     DF   
1    ENG-Premier League   2425  Arsenal      Bukayo Saka    ENG  MF,FW   
2    ENG-Premier League   2425  Arsenal       David Raya    ESP     GK   
3    ENG-Premier League   2425  Arsenal      Declan Rice    ENG     MF   
4    ENG-Premier League   2425  Arsenal    Ethan Nwaneri    ENG  MF,FW   
..                  ...    ...      ...              ...    ...    ...   
447  ENG-Premier League   2425   Wolves    Sam Johnstone    ENG     GK   
448  ENG-Premier League   2425   Wolves   Santiago Bueno    URU     DF   
449  ENG-Premier League   2425   Wolves      Tommy Doyle    ENG     MF   
450  ENG-Premier League   2425   Wolves       Toti Gomes    POR     DF   
451  ENG-Premier League   2425   Wolves  Yerson Mosquera    COL     DF   

        age  born Playing Time         ... Per 90 Minutes                    \
                            MP Starts  ...            Gls   Ast   G+A  G-PK   
0    27-025  1997            7      6  ...            0.0  0.15  0.15   0.0   
1    23-058  2001            8      8  ...           0.42  0.97  1.38  0.42   
2    29-048  1995            9      9  ...            0.0   0.0   0.0   0.0   
3    25-293  1999            8      8  ...            0.0  0.13  0.13   0.0   
4    17-226  2007            4      0  ...            0.0   0.0   0.0   0.0   
..      ...   ...          ...    ...  ...            ...   ...   ...   ...   
447  31-222  1993            5      5  ...            0.0   0.0   0.0   0.0   
448  25-359  1998            4      3  ...            0.0   0.0   0.0   0.0   
449  23-016  2001            7      1  ...            0.0  0.44  0.44   0.0   
450  25-291  1999            7      7  ...            0.0  0.15  0.15   0.0   
451  23-184  2001            5      5  ...            0.0   0.0   0.0   0.0   

                                              
    G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG  
0     0.15  0.02  0.03   0.04  0.02     0.04  
1     1.38  0.31  0.49    0.8  0.31      0.8  
2      0.0   0.0   0.0    0.0   0.0      0.0  
3     0.13  0.05  0.17   0.22  0.05     0.22  
4      0.0  0.07  0.35   0.42  0.07     0.42  
..     ...   ...   ...    ...   ...      ...  
447    0.0   0.0  0.01   0.01   0.0     0.01  
448    0.0  0.07   0.0   0.07  0.07     0.07  
449   0.44  0.12  0.04   0.16  0.12     0.16  
450   0.15   0.0  0.08   0.08   0.0     0.08  
451    0.0  0.07  0.02   0.09  0.07     0.09  

[452 rows x 37 columns]

In [99]:
player_stats500 = player_stats[player_stats["Playing Time"]['Min'] > 500]
player_stats500

league season     team             player nation    pos  \
                                                                           
0    ENG-Premier League   2425  Arsenal          Ben White    ENG     DF   
1    ENG-Premier League   2425  Arsenal        Bukayo Saka    ENG  MF,FW   
2    ENG-Premier League   2425  Arsenal         David Raya    ESP     GK   
3    ENG-Premier League   2425  Arsenal        Declan Rice    ENG     MF   
6    ENG-Premier League   2425  Arsenal  Gabriel Magalhães    BRA     DF   
..                  ...    ...      ...                ...    ...    ...   
440  ENG-Premier League   2425   Wolves       Mario Lemina    GAB     MF   
441  ENG-Premier League   2425   Wolves      Matheus Cunha    BRA  MF,FW   
443  ENG-Premier League   2425   Wolves      Nélson Semedo    POR     DF   
445  ENG-Premier League   2425   Wolves    Rayan Aït-Nouri    ALG     DF   
450  ENG-Premier League   2425   Wolves         Toti Gomes    POR     DF   

        age  born Playing Time         ... Per 90 Minutes                    \
                            MP Starts  ...            Gls   Ast   G+A  G-PK   
0    27-025  1997            7      6  ...            0.0  0.15  0.15   0.0   
1    23-058  2001            8      8  ...           0.42  0.97  1.38  0.42   
2    29-048  1995            9      9  ...            0.0   0.0   0.0   0.0   
3    25-293  1999            8      8  ...            0.0  0.13  0.13   0.0   
6    26-319  1997            9      9  ...           0.23   0.0  0.23  0.23   
..      ...   ...          ...    ...  ...            ...   ...   ...   ...   
440  31-062  1993            9      9  ...           0.12  0.12  0.24  0.12   
441  25-159  1999            9      8  ...           0.51   0.0  0.51  0.51   
443  30-352  1993            7      7  ...            0.0   0.3   0.3   0.0   
445  23-149  2001            9      9  ...           0.36  0.24  0.61  0.36   
450  25-291  1999            7      7  ...            0.0  0.15  0.15   0.0   

                                              
    G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG  
0     0.15  0.02  0.03   0.04  0.02     0.04  
1     1.38  0.31  0.49    0.8  0.31      0.8  
2      0.0   0.0   0.0    0.0   0.0      0.0  
3     0.13  0.05  0.17   0.22  0.05     0.22  
6     0.23  0.17  0.03    0.2  0.17      0.2  
..     ...   ...   ...    ...   ...      ...  
440   0.24  0.12  0.03   0.15  0.12     0.15  
441   0.51  0.28  0.23   0.51  0.28     0.51  
443    0.3  0.05  0.15   0.19  0.05     0.19  
445   0.61  0.13  0.12   0.25  0.13     0.25  
450   0.15   0.0  0.08   0.08   0.0     0.08  

[183 rows x 37 columns]

In [82]:
len(set(player_stats500['player']) & set(tm_player_stats500['player_name']))

171

In [83]:
# players unaccounted for in the primary dataset
set(player_stats500['player']) - set(tm_player_stats500['player_name'])

{'Emerson Palmieri',
 'Idrissa Gana Gueye',
 'Illia Zabarnyi',
 'Joško Gvardiol',
 'Jurriën Timber',
 'Mateo Kovačić',
 'Max Kilman',
 'Son Heung-min',
 'Tomáš Souček',
 'Toti Gomes',
 'Valentino Livramento',
 'Victor Bernth Kristiansen'}

In [84]:
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Emerson", "player_name"] = "Emerson Palmieri"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Idrissa Gueye", "player_name"] = "Idrissa Gana Gueye"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Ilya Zabarnyi", "player_name"] = "Illia Zabarnyi"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Josko Gvardiol", "player_name"] = "Joško Gvardiol"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Jurrien Timber", "player_name"] = "Jurriën Timber"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Mateo Kovacic", "player_name"] = "Mateo Kovačić"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Maximilian Kilman", "player_name"] = "Max Kilman"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Heung-min Son", "player_name"] = "Son Heung-min"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Tomas Soucek", "player_name"] = "Tomáš Souček"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Toti", "player_name"] = "Toti Gomes"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Tino Livramento", "player_name"] = "Valentino Livramento"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Victor Kristiansen", "player_name"] = "Victor Bernth Kristiansen"

In [85]:
set(player_stats500['player']) - set(tm_player_stats500['player_name'])

set()

In [87]:
tm_player_stats500 = tm_player_stats500.rename(columns={"player_name": "player"})
tm_player_stats500

,Unnamed: 0,team_name,league,country,player,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
0,1,Manchester City,Premier League,England,Ederson,https://www.transfermarkt.com/ederson/profil/s...,Goalkeeper,31,Brazil,48,37,0,3311
1,2,Manchester City,Premier League,England,Stefan Ortega,https://www.transfermarkt.com/stefan-ortega/pr...,Goalkeeper,32,Germany,57,21,0,1819
5,6,Manchester City,Premier League,England,Joško Gvardiol,https://www.transfermarkt.com/josko-gvardiol/p...,Left-Back,23,Croatia,56,53,6,4513
6,7,Manchester City,Premier League,England,Rúben Dias,https://www.transfermarkt.com/ruben-dias/profi...,Centre-Back,28,Portugal,43,41,0,3215
7,8,Manchester City,Premier League,England,Rico Lewis,https://www.transfermarkt.com/rico-lewis/profi...,Right-Back,20,England,54,43,2,3072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,785,Ipswich Town,Premier League,England,Jack Clarke,https://www.transfermarkt.com/jack-clarke/prof...,Left Winger,24,England,39,36,3,1481
786,787,Ipswich Town,Premier League,England,Conor Chaplin,https://www.transfermarkt.com/conor-chaplin/pr...,Centre-Forward,28,England,29,23,2,944
787,788,Ipswich Town,Premier League,England,Nathan Broadhead,https://www.transfermarkt.com/nathan-broadhead...,Left Winger,27,Wales,28,21,2,836
788,789,Ipswich Town,Premier League,England,George Hirst,https://www.transfermarkt.com/george-hirst/pro...,Centre-Forward,26,Scotland,29,29,5,833


In [103]:
player_stats500.columns.get_level_values(1)

Index(['', '', '', '', '', '', '', '', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls', 'Ast', 'G+A', 'G-PK',
       'G+A-PK', 'xG', 'xAG', 'xG+xAG', 'npxG', 'npxG+xAG'],
      dtype='object')

In [104]:
player_stats500.columns = player_stats.columns.get_level_values(1)
player_stats500

,,,,,,,,,MP,Starts,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,27-025,1997,7,6,...,0.0,0.15,0.15,0.0,0.15,0.02,0.03,0.04,0.02,0.04
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"MF,FW",23-058,2001,8,8,...,0.42,0.97,1.38,0.42,1.38,0.31,0.49,0.8,0.31,0.8
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,29-048,1995,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25-293,1999,8,8,...,0.0,0.13,0.13,0.0,0.13,0.05,0.17,0.22,0.05,0.22
6,ENG-Premier League,2425,Arsenal,Gabriel Magalhães,BRA,DF,26-319,1997,9,9,...,0.23,0.0,0.23,0.23,0.23,0.17,0.03,0.2,0.17,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,ENG-Premier League,2425,Wolves,Mario Lemina,GAB,MF,31-062,1993,9,9,...,0.12,0.12,0.24,0.12,0.24,0.12,0.03,0.15,0.12,0.15
441,ENG-Premier League,2425,Wolves,Matheus Cunha,BRA,"MF,FW",25-159,1999,9,8,...,0.51,0.0,0.51,0.51,0.51,0.28,0.23,0.51,0.28,0.51
443,ENG-Premier League,2425,Wolves,Nélson Semedo,POR,DF,30-352,1993,7,7,...,0.0,0.3,0.3,0.0,0.3,0.05,0.15,0.19,0.05,0.19
445,ENG-Premier League,2425,Wolves,Rayan Aït-Nouri,ALG,DF,23-149,2001,9,9,...,0.36,0.24,0.61,0.36,0.61,0.13,0.12,0.25,0.13,0.25


In [105]:
player_stats500.columns.values[0] = "league"
player_stats500.columns.values[1] = "season"
player_stats500.columns.values[2] = "team"
player_stats500.columns.values[3] = "player"
player_stats500.columns.values[4] = "nation"
player_stats500.columns.values[5] = "pos"
player_stats500.columns.values[6] = "age"
player_stats500.columns.values[7] = "born"
player_stats500

,league,season,team,player,nation,pos,age,born,MP,Starts,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,27-025,1997,7,6,...,0.0,0.15,0.15,0.0,0.15,0.02,0.03,0.04,0.02,0.04
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"MF,FW",23-058,2001,8,8,...,0.42,0.97,1.38,0.42,1.38,0.31,0.49,0.8,0.31,0.8
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,29-048,1995,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25-293,1999,8,8,...,0.0,0.13,0.13,0.0,0.13,0.05,0.17,0.22,0.05,0.22
6,ENG-Premier League,2425,Arsenal,Gabriel Magalhães,BRA,DF,26-319,1997,9,9,...,0.23,0.0,0.23,0.23,0.23,0.17,0.03,0.2,0.17,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,ENG-Premier League,2425,Wolves,Mario Lemina,GAB,MF,31-062,1993,9,9,...,0.12,0.12,0.24,0.12,0.24,0.12,0.03,0.15,0.12,0.15
441,ENG-Premier League,2425,Wolves,Matheus Cunha,BRA,"MF,FW",25-159,1999,9,8,...,0.51,0.0,0.51,0.51,0.51,0.28,0.23,0.51,0.28,0.51
443,ENG-Premier League,2425,Wolves,Nélson Semedo,POR,DF,30-352,1993,7,7,...,0.0,0.3,0.3,0.0,0.3,0.05,0.15,0.19,0.05,0.19
445,ENG-Premier League,2425,Wolves,Rayan Aït-Nouri,ALG,DF,23-149,2001,9,9,...,0.36,0.24,0.61,0.36,0.61,0.13,0.12,0.25,0.13,0.25


In [106]:
full_player_stats = player_stats500.merge(tm_player_stats500[["player", "player_pos"]], how="left", on="player")
full_player_stats

,league,season,team,player,nation,pos,age,born,MP,Starts,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,player_pos
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,27-025,1997,7,6,...,0.15,0.15,0.0,0.15,0.02,0.03,0.04,0.02,0.04,Right-Back
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"MF,FW",23-058,2001,8,8,...,0.97,1.38,0.42,1.38,0.31,0.49,0.8,0.31,0.8,Right Winger
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,29-048,1995,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goalkeeper
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25-293,1999,8,8,...,0.13,0.13,0.0,0.13,0.05,0.17,0.22,0.05,0.22,Central Midfield
4,ENG-Premier League,2425,Arsenal,Gabriel Magalhães,BRA,DF,26-319,1997,9,9,...,0.0,0.23,0.23,0.23,0.17,0.03,0.2,0.17,0.2,Centre-Back
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,ENG-Premier League,2425,Wolves,Mario Lemina,GAB,MF,31-062,1993,9,9,...,0.12,0.24,0.12,0.24,0.12,0.03,0.15,0.12,0.15,Defensive Midfield
180,ENG-Premier League,2425,Wolves,Matheus Cunha,BRA,"MF,FW",25-159,1999,9,8,...,0.0,0.51,0.51,0.51,0.28,0.23,0.51,0.28,0.51,Centre-Forward
181,ENG-Premier League,2425,Wolves,Nélson Semedo,POR,DF,30-352,1993,7,7,...,0.3,0.3,0.0,0.3,0.05,0.15,0.19,0.05,0.19,Right-Back
182,ENG-Premier League,2425,Wolves,Rayan Aït-Nouri,ALG,DF,23-149,2001,9,9,...,0.24,0.61,0.36,0.61,0.13,0.12,0.25,0.13,0.25,Left-Back


In [107]:
full_player_stats.to_csv("PL2425_500min_player_stats.csv")